In [1]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import SVD, NMF

# Зчитування даних з файлів
ratings_data = pd.read_csv('ratings.csv')
movies_data = pd.read_csv('movies.csv')
tags_data = pd.read_csv('tags.csv')
links_data = pd.read_csv('links.csv')

# Створення об'єкту 'Reader' для специфікації рейтингового діапазону
reader = Reader(rating_scale=(0.5, 5.0))

# Створення набору даних за допомогою 'Dataset.load_from_df'
data = Dataset.load_from_df(ratings_data[['userId', 'movieId', 'rating']], reader)

# Розділення даних на навчальний та тестовий набори
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Побудова моделі SVD
model = SVD()

# Навчання моделі на навчальному наборі
model.fit(trainset)

# Прогнозування рейтингів на тестовому наборі
predictions = model.test(testset)

# Виведення прикладів прогнозів
print("Приклади прогнозів моделі SVD:")
for prediction in predictions[:5]:
    was_impossible = prediction.details['was_impossible'] if 'was_impossible' in prediction.details else False
    print(f"user: {prediction.uid}\titem: {prediction.iid}\tr_ui = {prediction.r_ui:.2f}\test = {prediction.est:.2f}\twas_impossible: {was_impossible}")
print()

# Знаходження найкращих параметрів через крос-валідацію
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.2, 0.4, 0.6]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)

print("Найкращі параметри для SVD:")
print(grid_search.best_params['rmse'])

# Експериментування з іншими алгоритмами, наприклад, NMF
nmf_model = NMF()
nmf_model.fit(trainset)

print("Параметри для NMF:")
print(f"Кількість факторів: {nmf_model.n_factors}")


Приклади прогнозів моделі SVD:
user: 140	item: 6765	r_ui = 3.50	est = 3.49	was_impossible: False
user: 603	item: 290	r_ui = 4.00	est = 3.58	was_impossible: False
user: 438	item: 5055	r_ui = 4.00	est = 3.18	was_impossible: False
user: 433	item: 164179	r_ui = 5.00	est = 3.35	was_impossible: False
user: 474	item: 5114	r_ui = 4.00	est = 3.31	was_impossible: False

Найкращі параметри для SVD:
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}
Параметри для NMF:
Кількість факторів: 15


In [2]:
# Виведення матриці P (фактор користувача)
print("Матриця P (фактор користувача):")
print(nmf_model.pu)

Матриця P (фактор користувача):
[[0.70927076 0.5138997  0.4848116  ... 0.74738255 0.70496573 0.20245028]
 [0.43153542 0.36409166 0.56068172 ... 0.33723579 0.17175962 0.45334923]
 [0.58602107 0.62426218 0.25723104 ... 0.47537412 0.3226292  0.14909791]
 ...
 [0.67425808 0.5169372  0.08624513 ... 0.33571258 0.13611969 0.15180907]
 [1.02372769 0.15120175 0.19808917 ... 0.00740551 0.33326292 0.88725366]
 [0.56438801 0.36663787 0.71373119 ... 0.80117769 0.16118782 0.25198246]]


In [3]:
# Виведення матриці Q (фактор товару)
print("Матриця Q (фактор товару):")
print(nmf_model.qi)


Матриця Q (фактор товару):
[[1.04671372 0.11404915 0.85783962 ... 1.76205046 0.67922124 0.00852414]
 [0.38233342 0.44908433 0.57312748 ... 0.4445148  0.42616867 0.8056682 ]
 [0.85952542 0.35692434 0.3297837  ... 0.05313784 0.29845327 0.48883465]
 ...
 [0.78906153 0.2914976  0.49052604 ... 0.47064868 0.46626593 0.67340907]
 [0.60810829 1.02427813 0.67038336 ... 0.45555941 0.00552691 0.67509697]
 [0.52096003 0.70551761 0.50713088 ... 0.26292756 0.56933689 0.54121525]]


In [4]:
# Отримання факторів користувачів та товарів з моделі NMF
user_factors = nmf_model.pu
item_factors = nmf_model.qi

# Обчислення середніх рейтингів для кожного фактору користувача
avg_ratings_user = user_factors.mean(axis=1)
avg_ratings_item = item_factors.mean(axis=1)

# Вивід середніх рейтингів для кожного фактору користувача
print("Середній рейтинг для кожного фактору користувача:")
for i, avg_rating in enumerate(avg_ratings_user[:10]):
    print(f"Фактор користувача {i+1}: {avg_rating:.2f}")
print("...")

# Вивід середніх рейтингів для кожного фактору товару
print("Середній рейтинг для кожного фактору товару:")
for i, avg_rating in enumerate(avg_ratings_item[:10]):
    print(f"Фактор товару {i+1}: {avg_rating:.2f}")
print("...")

Середній рейтинг для кожного фактору користувача:
Фактор користувача 1: 0.48
Фактор користувача 2: 0.46
Фактор користувача 3: 0.40
Фактор користувача 4: 0.53
Фактор користувача 5: 0.42
Фактор користувача 6: 0.54
Фактор користувача 7: 0.50
Фактор користувача 8: 0.38
Фактор користувача 9: 0.50
Фактор користувача 10: 0.51
...
Середній рейтинг для кожного фактору товару:
Фактор товару 1: 0.43
Фактор товару 2: 0.49
Фактор товару 3: 0.44
Фактор товару 4: 0.49
Фактор товару 5: 0.53
Фактор товару 6: 0.31
Фактор товару 7: 0.50
Фактор товару 8: 0.60
Фактор товару 9: 0.33
Фактор товару 10: 0.52
...


In [5]:
import random

# Вибираємо 5 випадкових користувачів та 5 випадкових товарів
random_user_ids = random.sample(range(trainset.n_users), 5)
random_item_ids = random.sample(range(trainset.n_items), 5)

# Для кожного випадкового користувача та товару робимо прогноз рейтингу та виводимо його
for user_id in random_user_ids:
    for item_id in random_item_ids:
        # Здійснюємо прогноз рейтингу за допомогою моделі NMF для конкретного користувача та товару
        predicted_rating = nmf_model.predict(user_id, item_id).est
        # Виводимо інформацію про прогнозований рейтинг з двома десятковими знаками
        print(f"Користувач {user_id}, товар {item_id}: Прогнозований рейтинг = {predicted_rating:.2f}")


Користувач 347, товар 6951: Прогнозований рейтинг = 2.54
Користувач 347, товар 3336: Прогнозований рейтинг = 3.50
Користувач 347, товар 912: Прогнозований рейтинг = 4.49
Користувач 347, товар 3540: Прогнозований рейтинг = 3.50
Користувач 347, товар 1970: Прогнозований рейтинг = 3.44
Користувач 398, товар 6951: Прогнозований рейтинг = 2.42
Користувач 398, товар 3336: Прогнозований рейтинг = 3.50
Користувач 398, товар 912: Прогнозований рейтинг = 4.19
Користувач 398, товар 3540: Прогнозований рейтинг = 3.50
Користувач 398, товар 1970: Прогнозований рейтинг = 2.70
Користувач 291, товар 6951: Прогнозований рейтинг = 1.80
Користувач 291, товар 3336: Прогнозований рейтинг = 3.50
Користувач 291, товар 912: Прогнозований рейтинг = 4.11
Користувач 291, товар 3540: Прогнозований рейтинг = 3.50
Користувач 291, товар 1970: Прогнозований рейтинг = 3.61
Користувач 360, товар 6951: Прогнозований рейтинг = 3.60
Користувач 360, товар 3336: Прогнозований рейтинг = 3.50
Користувач 360, товар 912: Прогноз

In [6]:
# Створення сітки параметрів для NMF
nmf_param_grid = {'n_factors': [5, 10, 15], 'n_epochs': [50, 100, 150]}

# Пошук найкращих параметрів за допомогою крос-валідації для NMF
nmf_grid_search = GridSearchCV(NMF, nmf_param_grid, measures=['rmse'], cv=3)
nmf_grid_search.fit(data)

# Виведення результатів пошуку найкращих параметрів
print("Найкращі параметри для NMF:")
print(nmf_grid_search.best_params['rmse'])


Найкращі параметри для NMF:
{'n_factors': 15, 'n_epochs': 50}


In [7]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from surprise import SVD, NMF



# Порівняння різних моделей за різними метриками
print("Порівняння різних моделей:")

algorithm_names = ['SVD', 'NMF']  # Список назв алгоритмів для порівняння

metrics = ['rmse', 'mae']  # можна додати інші метрики

for metric in metrics:
    print(f"Метрика: {metric}")
    for algorithm in algorithm_names:
        model = None
        if algorithm == 'SVD':
            model = SVD()
        elif algorithm == 'NMF':
            model = NMF()
        cross_validate(model, data, measures=[metric], cv=3, verbose=True)

# Аналіз відмов та їх причин
print("Аналіз відмов:")
for prediction in predictions:
    was_impossible = prediction.details['was_impossible'] if 'was_impossible' in prediction.details else False
    if was_impossible:
        reason = prediction.details['reason'] if 'reason' in prediction.details else "Невідомо"
        print(f"Користувач {prediction.uid}, товар {prediction.iid}: Не можливо зробити прогноз. Причина: {reason}")


Порівняння різних моделей:
Метрика: rmse
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8752  0.8830  0.8843  0.8808  0.0040  
Fit time          0.50    0.53    0.51    0.51    0.01    
Test time         0.13    0.13    0.19    0.15    0.03    
Evaluating RMSE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9326  0.9369  0.9324  0.9339  0.0021  
Fit time          1.20    1.26    1.28    1.24    0.04    
Test time         0.17    0.19    0.18    0.18    0.01    
Метрика: mae
Evaluating MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     0.6773  0.6742  0.6775  0.6763  0.0015  
Fit time          0.51    0.52    0.51    0.51    0.01    
Test time         0.13    0.13    0.18    0.15    0.02    
Evaluating MAE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Me

In [8]:
test_size = len(testset)

# Зміна оцінки для 10% прогнозів, щоб створити відмови
num_failures = int(test_size * 0.1)  # 10% від розміру тестового набору
for i in range(num_failures):
    testset[i] = (testset[i][0], testset[i][1], 0.0)  # Змінюємо оцінку на 0.0

# Прогнозування рейтингів на тестовому наборі зі зміненими оцінками
predictions_with_failures = model.test(testset)

# Аналіз відмов та їх причин
print("Аналіз відмов:")
num_printed = 0
for prediction in predictions_with_failures:
    was_impossible = prediction.details['was_impossible'] if 'was_impossible' in prediction.details else False
    if was_impossible:
        reason = prediction.details['reason'] if 'reason' in prediction.details else "Невідомо"
        print(f"Користувач {prediction.uid}, товар {prediction.iid}: Не можливо зробити прогноз. Причина: {reason}")
        num_printed += 1
        if num_printed >= 10:  # Вивести лише 10 результатів
            break


Аналіз відмов:
Користувач 474, товар 6912: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 160, товар 2446: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 462, товар 152711: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 391, товар 695: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 606, товар 50356: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 599, товар 26913: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 387, товар 8196: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 6, товар 189: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 599, товар 101423: Не можливо зробити прогноз. Причина: User and item are unknown.
Користувач 474, товар 2757: Не можливо зробити прогноз. Причина: User and item are unknown.
